# Этап 04 Проверка работу pipeline

In [1]:
#!pip install catboost

In [2]:
import pickle
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt

/home/joefox/.pyvenv/base/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.preprocess import TrafficPreprocess

In [4]:
#Загрузим данные
df = pd.read_csv('data/part_10.csv.gz',compression='gzip')

In [5]:
df = df[:10]

In [6]:
#Инициализируем класс Препроцессинга и загрузим модели
traffic_preprocess = TrafficPreprocess()
traffic_preprocess.agent_model_load()
traffic_preprocess.load_models_etap_02()
traffic_preprocess.load_models_etap_03()
traffic_preprocess.load_models_etap_04()

In [7]:
#Выполним первый этап препроцессинга данных (выделения признаков)
list_result_dict = []
for idx,rows in tqdm(df.iterrows()):
    data = dict(rows)
    #print(data)
    result_dict = traffic_preprocess.preprocess_stage_one(data)
    list_result_dict.append(result_dict)
    #break

10it [00:00, 2025.55it/s]


In [8]:
#Загрузим модель кластеризации USER AGENT
traffic_preprocess.agent_model_load()

In [9]:
#Выполним второй этап препроцессинга данных (выделения признаков), и удалим записи с неверным IP
add_result_dict = []
for result_dict in tqdm(list_result_dict):
    result_dict = traffic_preprocess.preprocess_stage_two(result_dict)
    if result_dict['RIGHT_CLIENT_IP'] == result_dict['CLIENT_IP']:
        add_result_dict.append(result_dict)
    #break
list_result_dict.clear()
del(list_result_dict)

100%|██████████| 10/10 [00:00<00:00, 15.53it/s]


In [10]:
df = pd.DataFrame(add_result_dict)

In [11]:
#Сгенерируем итоговый массив признаков для кластеризации
list_futures = []
for result_dict in tqdm(add_result_dict):
    futures = traffic_preprocess.clusters_create_futures_array(result_dict)
    list_futures.append(futures)
    #break
# add_result_dict.clear()    
# del(add_result_dict)

100%|██████████| 10/10 [00:00<00:00, 12471.91it/s]


In [12]:
list_trafic_class = []
for futures in tqdm(list_futures):
    trafic_class = int(traffic_preprocess.trafic_model_predict(futures)[0])
    list_trafic_class.append(trafic_class)

100%|██████████| 10/10 [00:00<00:00, 542.66it/s]


In [13]:
type(trafic_class)

int

In [14]:
df = pd.DataFrame(add_result_dict)

In [15]:
df['trafic_class'] = list_trafic_class

In [16]:
#Оставим только аномальный трафик для кластеризации
df = df[df['trafic_class'] == 1]
df.reset_index(drop=True, inplace=True)
df.shape

(5, 20)

In [17]:
list_futures = []
for idx, rows in tqdm(df.iterrows()):
    result_dict = dict(rows)
    futures = traffic_preprocess.clusters_create_futures_array(result_dict)
    list_futures.append(futures)

5it [00:00, 219.11it/s]


In [18]:
df.head(3)

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID,RIGHT_CLIENT_IP,preproc_CLIENT_IP_country,preproc_CLIENT_USERAGENT,preproc_REQUEST_SIZE,preproc_RESPONSE_CODE_valid,preproc_RESPONSE_CODE,preproc_MATCHED_VARIABLE_SRC_valid,agent_vector,agent_cluster,matched_var_name,matched_var_value,trafic_class
0,188.138.92.55,NaN,166,404,REQUEST_URI,NaN,//tmp/20160925122692indo.php.vob,AVdhXFgVq1Ppo9zF5Fxu,188.138.92.55,DE,nan,166,True,404,True,"(0, 91)\t1.0",13,"[0.0227108, -0.02569362, 0.0186324, 0.02271437...","[0.02438902, -0.05339618, -0.007877337, -0.030...",1
1,176.123.240.64,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,395,403,REQUEST_PATH,NaN,"/opinion/page1_3.php\"" and \""\""x\""\""=\""\""x\""",iz7SN2YBrgKk_RFNZW_U,176.123.240.64,KG,mozilla/5.0 (windows nt 6.0; rv:34.0) gecko/20...,395,True,403,True,"(0, 149)\t0.2966918534009629\n (0, 121)\t0....",10,"[0.0227108, -0.02569362, 0.0186324, 0.02271437...","[0.0007681509, -0.001140485, -0.013785048, -0....",1
2,79.183.87.247,Mozilla/5.0 (Linux; Android 6.0; LG-H815 Build...,682,404,REQUEST_HEADERS,REQUEST_HEADERS.User-Agent,Mozilla/5.0 (Linux; Android 6.0; LG-H815 Build...,nFzwHGQB5cBXmMW1y_TD,79.183.87.247,IL,mozilla/5.0 (linux; android 6.0; lg-h815 build...,682,True,404,True,"(0, 122)\t0.16717495982469774\n (0, 87)\t0....",3,"[-0.010601818, -0.047821425, 0.016080828, -0.0...","[0.011902368, -0.019429851, -0.004136016, -0.0...",1


In [19]:
list_trafic_cluster = []
for futures in tqdm(list_futures):
    trafic_cluster = int(traffic_preprocess.trafic_cluster_predict(futures)[0])+2
    list_trafic_cluster.append(trafic_cluster)

100%|██████████| 5/5 [00:00<00:00, 1847.87it/s]


In [20]:
df['trafic_cluster'] = list_trafic_cluster

In [21]:
df.head(3)

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID,RIGHT_CLIENT_IP,preproc_CLIENT_IP_country,...,preproc_REQUEST_SIZE,preproc_RESPONSE_CODE_valid,preproc_RESPONSE_CODE,preproc_MATCHED_VARIABLE_SRC_valid,agent_vector,agent_cluster,matched_var_name,matched_var_value,trafic_class,trafic_cluster
0,188.138.92.55,NaN,166,404,REQUEST_URI,NaN,//tmp/20160925122692indo.php.vob,AVdhXFgVq1Ppo9zF5Fxu,188.138.92.55,DE,...,166,True,404,True,"(0, 91)\t1.0",13,"[0.0227108, -0.02569362, 0.0186324, 0.02271437...","[0.02438902, -0.05339618, -0.007877337, -0.030...",1,26
1,176.123.240.64,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,395,403,REQUEST_PATH,NaN,"/opinion/page1_3.php\"" and \""\""x\""\""=\""\""x\""",iz7SN2YBrgKk_RFNZW_U,176.123.240.64,KG,...,395,True,403,True,"(0, 149)\t0.2966918534009629\n (0, 121)\t0....",10,"[0.0227108, -0.02569362, 0.0186324, 0.02271437...","[0.0007681509, -0.001140485, -0.013785048, -0....",1,26
2,79.183.87.247,Mozilla/5.0 (Linux; Android 6.0; LG-H815 Build...,682,404,REQUEST_HEADERS,REQUEST_HEADERS.User-Agent,Mozilla/5.0 (Linux; Android 6.0; LG-H815 Build...,nFzwHGQB5cBXmMW1y_TD,79.183.87.247,IL,...,682,True,404,True,"(0, 122)\t0.16717495982469774\n (0, 87)\t0....",3,"[-0.010601818, -0.047821425, 0.016080828, -0.0...","[0.011902368, -0.019429851, -0.004136016, -0.0...",1,26
